In [1]:
                              ###############  LSTM_PYRO_V0 MEJORADA  ##################
    """
    MEJORAS IMPLEMENTADAS

    Manejo de excepciones:
        Se agregó manejo de excepciones en todas las funciones para evitar fallos inesperados.
        Se proporcionan valores predeterminados en caso de error.
    Mejoras en la estructura:
        Se creó una función run_demo() para ejecutar todo el proceso de demostración.
    Mejoras en la documentación:
        Se mejoraron los docstrings con más detalles y ejemplos.
    Flexibilidad:
        Se agregaron parámetros opcionales para personalizar el tamaño del tensor y la dimensión del modelo.
    Consistencia:
        Se utilizó tipado en todas las funciones.
    """
# Importamos las bibliotecas necesarias

import torch
import pyro
import pyro.distributions as dist
from typing import Tuple, Optional, Dict, Any

def check_system_requirements() -> Dict[str, Any]:
    """
    Verifica y recopila información sobre los requisitos del sistema.
    
    Returns:
        Dict[str, Any]: Diccionario con información del sistema
        
    Example:
        >>> info = check_system_requirements()
        >>> print(info['cuda_available'])
        True
    """
    try:
        info = {
            'cuda_available': torch.cuda.is_available(),
            'cuda_version': torch.version.cuda if torch.cuda.is_available() else None,
            'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
            'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
            'pyro_version': pyro.__version__,
            'pytorch_version': torch.__version__,
            'cudnn_available': torch.backends.cudnn.enabled if torch.cuda.is_available() else False,
            'cudnn_version': torch.backends.cudnn.version() if torch.cuda.is_available() and torch.backends.cudnn.enabled else None
        }
        return info
    except Exception as e:
        print(f"Error al verificar requisitos del sistema: {e}")
        # Devolvemos un diccionario con valores predeterminados en caso de error
        return {
            'cuda_available': False,
            'pyro_version': pyro.__version__,
            'pytorch_version': torch.__version__,
            'error': str(e)
        }

def print_system_info(info: Dict[str, Any]) -> None:
    """
    Imprime la información del sistema de manera formateada.
    
    Args:
        info (Dict[str, Any]): Diccionario con información del sistema
    """
    print("===== Información del Sistema =====")
    print(f"CUDA disponible: {info['cuda_available']}")
    if info['cuda_available']:
        print(f"Versión CUDA: {info['cuda_version']}")
        print(f"Número de GPUs: {info['gpu_count']}")
        print(f"GPU actual: {info['gpu_name']}")
        print(f"cuDNN disponible: {info['cudnn_available']}")
        if info['cudnn_available']:
            print(f"Versión cuDNN: {info['cudnn_version']}")
    print(f"Versión Pyro: {info['pyro_version']}")
    print(f"Versión PyTorch: {info['pytorch_version']}")
    if 'error' in info:
        print(f"Error encontrado: {info['error']}")
    print("=================================")

def setup_device() -> Tuple[torch.device, Optional[str]]:
    """
    Configura y devuelve el dispositivo de cómputo apropiado (GPU CUDA o CPU).
    
    Returns:
        Tuple[torch.device, Optional[str]]: Una tupla que contiene:
            - El dispositivo seleccionado (CUDA o CPU)
            - El nombre de la GPU si CUDA está disponible, None en caso contrario
    """
    try:
        # Verificamos si CUDA está disponible y configuramos el dispositivo
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Obtenemos el nombre de la GPU si usamos CUDA, None en caso contrario
        gpu_name = torch.cuda.get_device_name(0) if device.type == "cuda" else None
        return device, gpu_name
    except Exception as e:
        print(f"Error al configurar el dispositivo: {e}")
        return torch.device("cpu"), None

def create_example_tensor(device: torch.device, size: Tuple[int, int] = (3, 3)) -> torch.Tensor:
    """
    Crea un tensor aleatorio de ejemplo en el dispositivo especificado.
    
    Args:
        device (torch.device): El dispositivo donde crear el tensor
        size (Tuple[int, int], optional): Tamaño del tensor. Por defecto (3, 3)
    
    Returns:
        torch.Tensor: Un tensor aleatorio del tamaño especificado
    """
    try:
        # Generamos un tensor aleatorio en el dispositivo especificado
        return torch.rand(size, device=device)
    except Exception as e:
        print(f"Error al crear tensor: {e}")
        # En caso de error, devolvemos un tensor en CPU
        return torch.rand(size)

def simple_normal_model(device: torch.device, dim: int = 3) -> torch.Tensor:
    """
    Define y ejecuta un modelo Pyro simple que muestrea de una distribución normal multivariada.
    
    Args:
        device (torch.device): El dispositivo donde ejecutar el modelo
        dim (int, optional): Dimensión del vector. Por defecto 3
    
    Returns:
        torch.Tensor: Una muestra de la distribución normal multivariada
    """
    try:
        # Creamos un tensor de medias (loc) con ceros
        loc = torch.zeros(dim, device=device)
        # Creamos un tensor de escala con unos
        scale = torch.ones(dim, device=device)
        # Muestreamos de la distribución normal y convertimos a evento
        return pyro.sample("obs", dist.Normal(loc, scale).to_event(1))
    except Exception as e:
        print(f"Error en el modelo: {e}")
        # En caso de error, devolvemos un tensor en CPU
        loc = torch.zeros(dim)
        scale = torch.ones(dim)
        return pyro.sample("obs", dist.Normal(loc, scale).to_event(1))

# Función para ejecutar una demostración completa
def run_demo():
    """
    Ejecuta una demostración completa del entorno PyTorch/Pyro con CUDA.
    """
    # Verificamos los requisitos del sistema
    system_info = check_system_requirements()
    print_system_info(system_info)
    
    # Configuramos el dispositivo de cómputo
    device, gpu_name = setup_device()
    print(f"Dispositivo seleccionado: {device}")
    if gpu_name:
        print(f"Nombre de la GPU: {gpu_name}")
    
    # Creamos e imprimimos un tensor de ejemplo
    tensor = create_example_tensor(device)
    print(f"Tensor en dispositivo: {tensor}")
    
    # Generamos e imprimimos una muestra de la distribución normal
    muestra = simple_normal_model(device)
    print(f"Muestra del modelo: {muestra}")
    
    return {
        'system_info': system_info,
        'device': device,
        'tensor': tensor,
        'muestra': muestra
    }

# Bloque principal de ejecución
if __name__ == "__main__":
    run_demo()

===== Información del Sistema =====
CUDA disponible: True
Versión CUDA: 11.8
Número de GPUs: 1
GPU actual: NVIDIA GeForce RTX 4050 Laptop GPU
cuDNN disponible: True
Versión cuDNN: 90100
Versión Pyro: 1.9.1
Versión PyTorch: 2.6.0+cu118
Dispositivo seleccionado: cuda
Nombre de la GPU: NVIDIA GeForce RTX 4050 Laptop GPU
Tensor en dispositivo: tensor([[0.7199, 0.8336, 0.5788],
        [0.7577, 0.7110, 0.9808],
        [0.7698, 0.7932, 0.4157]], device='cuda:0')
Muestra del modelo: tensor([-1.5095,  0.2916, -0.0107], device='cuda:0')
